# Selective Transformer Model

Create a transformer model with heriarchical self attention and a loss function that optimizes selective risk, or create a model g(x) that predicts the risk of a given input and rejects inputs with risk above a threshold. The g(x) model can possibly take attention heads as input?

The goal of this model is to not make predictions on inputs that we aren't confident will be higher or lower than the cross-sectional median, (we can adjust for a different objective like return buckets later). We also want to stochastic self-attention to learn time-series patterns in the data, and add stochasticity to better generalize to unseen data. The model g(x) should take 

Question for Kumar: It seems like incorperating stochastic self-attention, we need to make a loss function specific to returns, and calculate the self-attention relative to what we're predicting.

Data Transformations
- Normalized Returns
- Log Normalized Returns
- Return Derivatives relative to factor models
- Wavelet Transform 

Stochastic self-attention Mechanisms:
- Grumbel-softmax: https://assets.amazon.science/1a/48/cb3245fb448ba775f163f02c2e6b/transformer-uncertainty-estimation-with-hierarchical-stochastic-attention.pdf
    - In forward function of transformer replace softmax with grumbel-softmax. Leading to stochastic attention heads.
- Wasserstein distance: https://www.youtube.com/watch?v=5Lrs880Z7VQ
    - For wasserstien distance they use the item-item distance in sequences to predict. So someone listens to Nirvana, and then Foot Fighters. How would we adapt this mechanism to returns, clustering stocks would be an interesting application, but for returns prediction are we looking at the wasserstein distance between days, or each stock relative to the cross-sectional median?
    - In the paper they rank reccomendations based off wasserstein distance score, we can rank confidence of predictions based off wasserstein distance score, and then rank based off return bucket. 
    - Create mean and covaraince embeddings from previous steps to sample the next step. 
- Novel approach? 
    - It seems like we need a novel approach to inject stochasticity into our Transformer model that will fully relate to our loss function. Again the goal is to better generalize time-series patterns, and to not make predictions on inputs that we aren't confident will be higher or lower than the cross-sectional median, or whatever our loss function ends up being. 
    - Also can train a first layer model to filter out non confident stocks, and then use transformer to predict returns on the remaining stocks.

Loss Functions:
- Selective Risk: https://arxiv.org/pdf/1901.09192.pdf
- Sharpe ratio (would need to modify input data to be split into train/test and train by day (average returns and calculate sharpe each day. Can probably batch days of data together to speed up training))
- Raw Returns
- Hi/Lo Cross-sectional median

The Deep Learning Stat Arb paper shows that Transformers are really good out figuring at time-series patterns, they use a CNN to simplify time-series patterns to use as input into the Transformer. https://arxiv.org/pdf/2106.04028.pdf

![Alt text](image.png)

Is this a transformation we should consider, or is a simplified objective function of hi/lo cross-sectional median enough? My curisosity is that using just the cross-sectional median is too simple of a loss function to take full advatange of the transformer. Will the transformer learn time-series patterns and data distrubtions from just predicting the Hi/Lo the cross-sectional median?

We also want to have a mechandism to reject an input x. Whether this is a confidence threshold according to our attention mechandism, or a model g(x) that predicts the risk of a given input and rejects inputs with risk above a threshold. The g(x) model can possibly take attention heads as input? Also should we incorperate rejection into our loss function?

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

class StochasticSelfAttention(nn.Module):
    def __init__(self,embedding_size,heads):
        super(StochasticSelfAttention,self).__init__()
        self.embedding_size = embedding_size
        self.heads = heads
        self.head_dim = embedding_size // heads
        self.values = nn.Linear(self.head_dim,self.head_dim,bias=False)
        self.keys = nn.Linear(self.head_dim,self.head_dim,bias=False)
        self.queries = nn.Linear(self.head_dim,self.head_dim,bias=False)
        self.fc_out = nn.Linear(embedding_size,embedding_size)